<h1 align =center><font color =#ff3333>Battle of Suitable Neighbourhoods </font></h1>

<h1><font color =red>Target City</font></h1>

**Bangalore**, officially known as *Bengaluru*, is the capital of the Indianstate of Karnataka. It has a population of over ten million,making it a megacity and the third-most populous city and fifth-most populous urban agglomeration in India.Bangalore is the second fastest-growing major metropolis in India. 

Bengaluru has one of the most highly educated workforces in the world. The city attracts thousands of software engineers and IT specialists from all over India. Bengaluru is not just an IT Hub but it is also the hub of other industries like aeronautics, space research and defence research.

<h2><font color =brown> 1. Description of the Problem</font></h2>

**Bangalore** is known as the Silicon Valley of India. With the increasing number of tech comapnies and startups, a lot of people are shifting to Bengaluru. People faces a lot of problems when they shift from one city to another  like which is the ideal place that is near to Mall, Hospitals , restaurants, parks, metro station etc. 

So, as a part of this project we will find the best place to live which near to metro, Hospitals and malls.

<h1><font color =blue>2. Data Section</font></h1>

For this project we need the following data:
1. Bangalore data that contain neighbourhoods.
    * Data Source:https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore
    * Description: This data contain neighbourhoods name of Bangalore. As we will use to explore venues in each  neighbourhoods.We can filter these venues to get only resturants, malls and Parks. 
2. Bangalore Metro Stations and Hospitals data. 
    * Data Source:FourSquare API
    * Description:By using this API we will get all the location of metro stations and hospitals.
  

# Approach

* Collect the Bangalore Neighbourhood  data from https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore
* Using Foursquare API we will get Metro Stations in Bengalore.
* AS well as we will also get Hospitals in Bengalore.
* For required venues we also using Foursquare API we will get all venues for each neighborhood.
* Filter out all venues which are Shopping Malls, Parks and Restaurants.
* Data Visualization and some statistical analysis.
* Analysing using Clustering (Specially K-Means):
    1. Find the best value of K 
    2. Visualize the neighborhood with metro and hospitals location.
* Compare the Neighborhoods to Find the Best Place for living in Bangalore
* Inference From these Results and related Conclusions

In [1]:
# Import the relevant libraries
import pandas as pd  # library for data analsysis
import numpy as np
!conda install -c conda-forge geopy
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
import requests
import matplotlib.pyplot as plt
!conda install -c conda-forge folium=0.5.0 --yes
import folium  # map rendering library
import json # library to handle JSON files
from sklearn.cluster import KMeans  # import k-means from clustering stage
# pd.options.display.max_columns = None
pd.options.display.max_rows =None
pd.options.display.max_columns =None
print('Done')

#from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Download and Explore Dataset
    1.1 Data of Bangalore Neighbourhoods
    1.2 Data of Bangalore Metro Stations or Subways
    1.3 Data of Bangalore Hospitals
    1.4 Data of target venue i.e. Parks, Shopping Mall and Restaurant.

## Define Foursquare Credentials and Version

In [3]:
CLIENT_ID = 'Your client ID'
CLIENT_SECRET = 'Your client secret ID '
VERSION = '20191008'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T4LAOXFRO1543XC3IS4Q5SO3VL5S5DPLZNSW0Z2IYCUKIAQ2
CLIENT_SECRET:LCFYXDNEEOAU35H5PEA351QTNUGPXTMIVNL3C1PKJ40X0IZG


### Let's get the geographical coordinates of Bangalore.

In [5]:
address = 'Bangalore'

geolocator = Nominatim(user_agent = 'foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


## 1.1 Data of Bangalore Neighbourhoods

In [6]:
df_neighbourhood = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore',match ='Name') # use read_html method to read tables on webpage
df_neighbourhood[0] # first table of that webpage

,Name,Image,Summary
0,Cantonment area,NaN,The Cantonment area in Bangalore was used as a...
1,Domlur,NaN,"Formerly part of the Cantonment area, Domlur h..."
2,Indiranagar,NaN,Indiranagar is a sought-after residential and ...
3,Jeevanbheemanagar,NaN,NaN
4,Malleswaram,NaN,NaN
5,Pete area,NaN,Established by Kempe Gowda I at the time of cr...
6,Sadashivanagar,NaN,Sadashivanagar is an upscale neighbourhood in ...
7,Seshadripuram,NaN,Seshadripuram was established in 1892 to reduc...
8,Shivajinagar,NaN,Shivajinagar is one of the older areas of the ...
9,Ulsoor,NaN,Ulsoor (or Halasuru) is one of the oldest area...


In [7]:
#  Similarly, there are 7 more tables. Let combine into one dataframe--
df_neighbourhood = pd.concat([df_neighbourhood[0],df_neighbourhood[1],df_neighbourhood[2],df_neighbourhood[3],df_neighbourhood[4],
                              df_neighbourhood[5],df_neighbourhood[6],df_neighbourhood[7]], ignore_index =True)
df_neighbourhood 

,Name,Image,Summary
0,Cantonment area,NaN,The Cantonment area in Bangalore was used as a...
1,Domlur,NaN,"Formerly part of the Cantonment area, Domlur h..."
2,Indiranagar,NaN,Indiranagar is a sought-after residential and ...
3,Jeevanbheemanagar,NaN,NaN
4,Malleswaram,NaN,NaN
5,Pete area,NaN,Established by Kempe Gowda I at the time of cr...
6,Sadashivanagar,NaN,Sadashivanagar is an upscale neighbourhood in ...
7,Seshadripuram,NaN,Seshadripuram was established in 1892 to reduc...
8,Shivajinagar,NaN,Shivajinagar is one of the older areas of the ...
9,Ulsoor,NaN,Ulsoor (or Halasuru) is one of the oldest area...


In [8]:
# drop the image and Summary columns from dataframe
df_neighbourhood.drop(['Image','Summary'], axis=1, inplace=True)

In [9]:
df_neighbourhood.shape # size of the dataframe

(65, 1)

In [10]:
# Let find the location of each Neighbourhood of Bangalore
df_neighbourhood.replace(['Jeevanbheemanagar'],['Jeevan bheema nagar'],inplace =True)# change the name to get coordinate.

latitude_b = [] #latitude list of nighbourhoods
longitude_b= [] #latitude list of nighbourhoods
for i in df_neighbourhood['Name']:
    try:
        address = i + ', Bangalore'

        geolocator = Nominatim(user_agent = 'bangalore_explorer')
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    #     print(latitude, longitude)
        latitude_b.append(latitude)
        longitude_b.append(longitude)
    except:
        latitude_b.append(None)
        longitude_b.append(None)

In [11]:
# assign the latitude and longitude values in each Neighbourhood dataframe.
df_neighbourhood['Latitude'] = latitude_b
df_neighbourhood['Longitude'] = longitude_b

#rename the column name of Name to Neighbourhood
df_neighbourhood.rename(columns={'Name':'Neighbourhood'}, inplace =True)

In [12]:
df_neighbourhood

,Neighbourhood,Latitude,Longitude
0,Cantonment area,13.019567,77.509589
1,Domlur,12.962467,77.638196
2,Indiranagar,12.973291,77.640467
3,Jeevan bheema nagar,12.969316,77.653668
4,Malleswaram,13.016341,77.558664
5,Pete area,NaN,NaN
6,Sadashivanagar,13.007708,77.579589
7,Seshadripuram,12.993188,77.575342
8,Shivajinagar,12.986391,77.607542
9,Ulsoor,12.977879,77.624670


In [13]:
# Drop the Null values in Neighbourhood dataframe
df_neighbourhood.dropna(subset=['Latitude'], inplace=True)

In [14]:
df_neighbourhood[df_neighbourhood.isnull().values] #check any null value in a dataframe

,Neighbourhood,Latitude,Longitude


In [15]:
df_neighbourhood.reset_index(drop =True, inplace =True)# reset the index of dataframe
df_neighbourhood.head()

,Neighbourhood,Latitude,Longitude
0,Cantonment area,13.019567,77.509589
1,Domlur,12.962467,77.638196
2,Indiranagar,12.973291,77.640467
3,Jeevan bheema nagar,12.969316,77.653668
4,Malleswaram,13.016341,77.558664


### Map :- Neighbourhoods of Bangalore

In [16]:
map_bang = folium.Map(location =[latitude, longitude], zoom_start =12) # create map of Bangalore using latitude and longitude values

# add markers to map
for lat, lng, neighbourhood in zip(df_neighbourhood['Latitude'],df_neighbourhood['Longitude'],df_neighbourhood['Neighbourhood']):
    label ='{}'.format(neighbourhood)
    label =folium.Popup(label, parse_html= True)
    folium.CircleMarker(
        [lat,lng],
        radius = 6,
        popup =label,
        color = 'black',
        fill =True,
        fill_color = '#ffff00',
        fill_opacity =0.9).add_to(map_bang)
map_bang

## 1.2 Data of Bangalore Metro Stations and Subways